# Equal-Weight S&P 500 Index Fund

## Introduction & Library Imports

The S&P 500 is the world's most popular stock market index. The largest fund that is benchmarked to this index is the SPDR® S&P 500® ETF Trust. It has more than US$250 billion of assets under management.

The goal of this section of the course is to create a Python script that will accept the value of your portfolio and tell you how many shares of each S&P 500 constituent you should purchase to get an equal-weight version of the index fund.

## Library Imports

The first thing we need to do is import the open-source software libraries that we'll be using in this tutorial.

In [41]:
import numpy as np
import pandas as pd
import requests
import xlsxwriter
import math

## Importing Our List of Stocks

The next thing we need to do is import the constituents of the S&P 500.

These constituents change over time, so in an ideal world you would connect directly to the index provider (Standard & Poor's) and pull their real-time constituents on a regular basis.

Paying for access to the index provider's API is outside of the scope of this course. 

There's a static version of the S&P 500 constituents available here. [Click this link to download them now](https://drive.google.com/file/d/1ZJSpbY69DVckVZlO9cC6KkgfSufybcHN/view?usp=sharing). Move this file into the `starter-files` folder so it can be accessed by other files in that directory.

Now it's time to import these stocks to our Jupyter Notebook file.

In [42]:
stocks = pd.read_csv("sp_500_stocks.csv")

In [43]:
from secrets import IEX_CLOUD_API_TOKEN

## Making Our First API Call

Now it's time to structure our API calls to IEX cloud. 

We need the following information from the API:

* Market capitalization for each stock
* Price of each stock



In [44]:
symbol="AAPL"
api_url="https://cloud.iexapis.com/stable"
later_url=f"/stock/{symbol}/quote?token={IEX_CLOUD_API_TOKEN}"
data=requests.get(api_url+later_url).json()
print(data)

{'avgTotalVolume': 93375466, 'calculationPrice': 'close', 'change': -1.92, 'changePercent': -0.01172, 'close': 161.84, 'closeSource': 'official', 'closeTime': 1638565201109, 'companyName': 'Apple Inc', 'currency': 'USD', 'delayedPrice': 161.89, 'delayedPriceTime': 1638565197919, 'extendedChange': -0.59, 'extendedChangePercent': -0.00365, 'extendedPrice': 161.25, 'extendedPriceTime': 1638579599836, 'high': 164.96, 'highSource': '15 minute delayed price', 'highTime': 1638565200116, 'iexAskPrice': None, 'iexAskSize': None, 'iexBidPrice': None, 'iexBidSize': None, 'iexClose': 161.91, 'iexCloseTime': 1638565197063, 'iexLastUpdated': None, 'iexMarketPercent': None, 'iexOpen': 164.015, 'iexOpenTime': 1638541800133, 'iexRealtimePrice': None, 'iexRealtimeSize': None, 'iexVolume': None, 'lastTradeTime': 1638565229890, 'latestPrice': 161.84, 'latestSource': 'Close', 'latestTime': 'December 3, 2021', 'latestUpdate': 1638565201109, 'latestVolume': 118023116, 'low': 159.72, 'lowSource': '15 minute d

## Parsing Our API Call

The API call that we executed in the last code block contains all of the information required to build our equal-weight S&P 500 strategy. 

With that said, the data isn't in a proper format yet. We need to parse it first.

In [45]:
price=data['latestPrice']
mcap=data['marketCap']

## Acquiring an API Token

Now it's time to import our IEX Cloud API token. This is the data provider that we will be using throughout this course.

API tokens (and other sensitive information) should be stored in a `secrets.py` file that doesn't get pushed to your local Git repository. We'll be using a sandbox API token in this course, which means that the data we'll use is randomly-generated and (more importantly) has no cost associated with it.

[Click here](http://nickmccullum.com/algorithmic-trading-python/secrets.py) to download your `secrets.py` file. Move the file into the same directory as this Jupyter Notebook before proceeding.

## Adding Our Stocks Data to a Pandas DataFrame

The next thing we need to do is add our stock's price and market capitalization to a pandas DataFrame. Think of a DataFrame like the Python version of a spreadsheet. It stores tabular data.

In [46]:
my_columns = ['Ticker','Stock Price','Market Cap','Number of Shares to Buy']

In [47]:
resultant_dataframe = pd.DataFrame(columns=my_columns)

## Looping Through The Tickers in Our List of Stocks

Using the same logic that we outlined above, we can pull data for all S&P 500 stocks and store their data in the DataFrame using a `for` loop.

In [48]:
for stock in stocks['Ticker']:
    symbol=stock
    url=api_url+later_url
    data =  requests.get(url).json()
    resultant_dataframe = resultant_dataframe.append(
        pd.Series([symbol,data['latestPrice'],data['marketCap'],'N/A'],index=my_columns),ignore_index=True
    )

In [49]:
print(resultant_dataframe)

    Ticker  Stock Price     Market Cap Number of Shares to Buy
0        A       161.84  2655211290480                     N/A
1      AAL       161.84  2655211290480                     N/A
2      AAP       161.84  2655211290480                     N/A
3     AAPL       161.84  2655211290480                     N/A
4     ABBV       161.84  2655211290480                     N/A
..     ...          ...            ...                     ...
500    YUM       161.84  2655211290480                     N/A
501    ZBH       161.84  2655211290480                     N/A
502   ZBRA       161.84  2655211290480                     N/A
503   ZION       161.84  2655211290480                     N/A
504    ZTS       161.84  2655211290480                     N/A

[505 rows x 4 columns]


## Using Batch API Calls to Improve Performance

Batch API calls are one of the easiest ways to improve the performance of your code.

This is because HTTP requests are typically one of the slowest components of a script.

Also, API providers will often give you discounted rates for using batch API calls since they are easier for the API provider to respond to.

IEX Cloud limits their batch API calls to 100 tickers per request. Still, this reduces the number of API calls we'll make in this section from 500 to 5 - huge improvement! In this section, we'll split our list of stocks into groups of 100 and then make a batch API call for each group.

In [50]:
def chunks(lst,n):
    for i in range(0,len(lst),n):
        yield lst[i:i+n]
        
symbol_groups = list(chunks(stocks['Ticker'],100))

print(symbol_groups)

symbol_strings = []

for i in range(len(symbol_groups)):
    symbol_strings.append(','.join(symbol_groups[i]))
    
print(symbol_strings)

[0         A
1       AAL
2       AAP
3      AAPL
4      ABBV
      ...  
95     CINF
96       CL
97      CLX
98      CMA
99    CMCSA
Name: Ticker, Length: 100, dtype: object, 100     CME
101     CMG
102     CMI
103     CMS
104     CNC
       ... 
195    FTNT
196     FTV
197      GD
198      GE
199    GILD
Name: Ticker, Length: 100, dtype: object, 200     GIS
201      GL
202     GLW
203      GM
204    GOOG
       ... 
295     MAA
296     MAR
297     MAS
298     MCD
299    MCHP
Name: Ticker, Length: 100, dtype: object, 300     MCK
301     MCO
302    MDLZ
303     MDT
304     MET
       ... 
395     RHI
396     RJF
397      RL
398     RMD
399     ROK
Name: Ticker, Length: 100, dtype: object, 400     ROL
401     ROP
402    ROST
403     RSG
404     RTX
       ... 
495    XLNX
496     XOM
497    XRAY
498     XRX
499     XYL
Name: Ticker, Length: 100, dtype: object, 500     YUM
501     ZBH
502    ZBRA
503    ZION
504     ZTS
Name: Ticker, dtype: object]
['A,AAL,AAP,AAPL,ABBV,ABC,ABMD,ABT,ACN,A

In [51]:
resultant_dataframe = pd.DataFrame(columns=my_columns)

for symbol_string in symbol_strings:
    batch_api_call = f"/stock/market/batch?symbols={symbol_string}&types=quote&token={IEX_CLOUD_API_TOKEN}"
    data=requests.get(api_url+batch_api_call).json()
    #print(batch_api_call+"\n")
    #print(data)
    
    for symbol in symbol_string.split(','):
        resultant_dataframe = resultant_dataframe.append(
            pd.Series(
                [
                    symbol,
                    data[symbol]['quote']['latestPrice'],
                    data[symbol]['quote']['marketCap'],
                    'N/A'
                ],index=my_columns
            ),ignore_index=True
        )

print(resultant_dataframe)

    Ticker  Stock Price     Market Cap Number of Shares to Buy
0        A       149.33    45205576012                     N/A
1      AAL        16.62    10760749151                     N/A
2      AAP       226.86    14145895454                     N/A
3     AAPL       161.84  2655211290480                     N/A
4     ABBV       118.85   210028811978                     N/A
..     ...          ...            ...                     ...
500    YUM       124.83    36591805247                     N/A
501    ZBH       121.22    25323828730                     N/A
502   ZBRA       588.29    31439094740                     N/A
503   ZION        61.91     9686652994                     N/A
504    ZTS       223.33   105663189827                     N/A

[505 rows x 4 columns]


## Calculating the Number of Shares to Buy

As you can see in the DataFrame above, we stil haven't calculated the number of shares of each stock to buy.

We'll do that next.

In [52]:
portfolio_size=input("Enter the value you want to invest")
try:
    val=float(portfolio_size)
except ValueError:
    print("Please Enter an Integer/Numeric Value")
    
position_size = val/len(resultant_dataframe.index)
print(position_size)

Enter the value you want to invest140000
277.2277227722772


In [53]:
for i in range(len(resultant_dataframe.index)):
    resultant_dataframe.loc[i,"Number of Shares to Buy"] = math.floor(position_size/resultant_dataframe.loc[i,"Stock Price"])
    
print(resultant_dataframe)

    Ticker  Stock Price     Market Cap Number of Shares to Buy
0        A       149.33    45205576012                       1
1      AAL        16.62    10760749151                      16
2      AAP       226.86    14145895454                       1
3     AAPL       161.84  2655211290480                       1
4     ABBV       118.85   210028811978                       2
..     ...          ...            ...                     ...
500    YUM       124.83    36591805247                       2
501    ZBH       121.22    25323828730                       2
502   ZBRA       588.29    31439094740                       0
503   ZION        61.91     9686652994                       4
504    ZTS       223.33   105663189827                       1

[505 rows x 4 columns]


## Formatting Our Excel Output

We will be using the XlsxWriter library for Python to create nicely-formatted Excel files.

XlsxWriter is an excellent package and offers tons of customization. However, the tradeoff for this is that the library can seem very complicated to new users. Accordingly, this section will be fairly long because I want to do a good job of explaining how XlsxWriter works.

### Initializing our XlsxWriter Object

In [54]:
writer = pd.ExcelWriter('recommended_allocations.xlsx',engine='xlsxwriter')

resultant_dataframe.to_excel(writer, "Recommended Allocations",index = False)
background_color = '#0a0a23'
font_color = '#ffffff'

string_format = writer.book.add_format(
    {
        'font_color':font_color,
        'bg_color':background_color,
        'border':1
    }
)

dollar_format = writer.book.add_format(
    {
        'num_format':'$0.00',
        'font_color':font_color,
        'bg_color':background_color,
        'border':1
    }
)

integer_format = writer.book.add_format(
    {
        'num_format':'0',
        'font_color':font_color,
        'bg_color':background_color,
        'border':1
    }
)

### Creating the Formats We'll Need For Our `.xlsx` File

Formats include colors, fonts, and also symbols like `%` and `$`. We'll need four main formats for our Excel document:
* String format for tickers
* \\$XX.XX format for stock prices
* \\$XX,XXX format for market capitalization
* Integer format for the number of shares to purchase

In [55]:
column_format = {
    'A':['Ticker',string_format],
    'B':['Ticker',dollar_format],
    'C':['Ticker',dollar_format],
    'D':['Ticker',integer_format],
}

### Applying the Formats to the Columns of Our `.xlsx` File

We can use the `set_column` method applied to the `writer.sheets['Recommended Trades']` object to apply formats to specific columns of our spreadsheets.

Here's an example:

```python
writer.sheets['Recommended Trades'].set_column('B:B', #This tells the method to apply the format to column B
                     18, #This tells the method to apply a column width of 18 pixels
                     string_template #This applies the format 'string_template' to the column
                    )
```

In [56]:
for column in column_format.keys():
    writer.sheets['Recommended Allocations'].set_column(f"{column}:{column}",18,column_format[column][1])

This code works, but it violates the software principle of "Don't Repeat Yourself". 

Let's simplify this by putting it in 2 loops:

## Saving Our Excel Output

Saving our Excel file is very easy:

In [57]:
writer.save()